In [1]:
__version__ = '0.1'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [3]:
#import pydecode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ImportError: dlopen(//anaconda/lib/python2.7/site-packages/matplotlib/ft2font.so, 2): Library not loaded: @loader_path/../../../libfreetype.6.dylib
  Referenced from: //anaconda/lib/python2.7/site-packages/matplotlib/ft2font.so
  Reason: Incompatible library version: ft2font.so requires version 18.0.0 or later, but libfreetype.6.dylib provides version 13.0.0

Die normale Modellformulierung des Auftragsannahmeproblems im Revenue Management von Instandhaltungsprozessen:

$$V(\textbf{c}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max(V(\textbf{c}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, t-1)) + p_{0}(t)V(\textbf{c}, t-1) $$

$$= V(\textbf{c}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)(r_j - V(\textbf{c}, t-1) + V(\textbf{c}-\textbf{a}_j, t-1))$$

Eine Anfrage kann akzeptiert oder abgelehnt werden:

In [9]:
#kReject, kAccept = 0, 1
#operations = np.array([kReject, kAccept])
#offsets = np.array([[1,0], [0,1], [1,1]]) 
#op_names = np.array(["Reject",  "Accept"])

Es gibt $j$ Produkte:

In [10]:
def Product(j):
    return np.arange(j+1)

products = Product(3)
products

array([0, 1, 2, 3])

Es gibt $h$ Ressourcen:

In [11]:
def Resource(h):
    return np.arange(h+1)

resources = Resource(2)
resources

array([0, 1, 2])

In [12]:
def Capacity(h):
    matrix = np.zeros(shape=(len(h)))
    return matrix

capacities = Capacity(resources)
capacities[1] = 5
capacities[2] = 4
capacities

array([ 0.,  5.,  4.])

Eine Matrix des Ressourcenverbrauchs $a_{j}$ für jedes Produkte $j$:

In [13]:
def Consumption(j, h):
    matrix = np.zeros(shape=(len(j),len(h)))
    #matrix[0] = [range]
    return matrix

consumtions = Consumption(products, resources)
consumtions[1] = [0,1,0]
consumtions[2] = [0,0,1]
consumtions[3] = [0,1,1]
consumtions

array([[ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  1.]])

Jedes Produkt $j$ erzielt einen Ertrag $r_j$:

In [14]:
def Revenue(r):
    vector = np.zeros(shape=(len(r)))
    return vector

revenues = Revenue(products)
revenues[1] = 100
revenues[2] = 200
revenues[3] = 300
revenues

array([   0.,  100.,  200.,  300.])

Eine Anzahl an Buchungsperioden $t$ (Buchungshorizont):

In [15]:
def Time(t):
    time = np.arange(t+1)
    time = time[::-1]
    return time

times = Time(4)
times

array([4, 3, 2, 1, 0])

Eine Anfragen nach einem Produkte $j$ zum Zeitpunkt $t$ hat eine bestimmte Eintrittfahrscheinlichkeit $p_j(t)$:

In [16]:
def Prob(j, t):
    matrix = np.zeros(shape=(len(j),len(t)))
    #matrix[0] = [range]
    return matrix

probs = Prob(products, times)
probs[1] = [0.4, 0.3, 0.2, 0.1, 0.]
probs[2] = [0.3, 0.2, 0.1, 0.2, 0.]
probs[3] = [0.2, 0.1, 0.2, 0.3, 0.]
probs

array([[ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.4,  0.3,  0.2,  0.1,  0. ],
       [ 0.3,  0.2,  0.1,  0.2,  0. ],
       [ 0.2,  0.1,  0.2,  0.3,  0. ]])

Die Wahrscheinlichkeit, dass keine Anfrage eintrifft entspricht $p_0(t)$

In [17]:
def Against_Prob(prob):
    matrix = prob.T
    matrix2 = np.zeros(shape=(len(matrix)))
    for i in range(0, len(matrix)):
        matrix2[i] = 1-(matrix[i].sum())
    return matrix2
    
against_probs = Against_Prob(probs)
against_probs

array([ 0.1,  0.4,  0.5,  0.4,  1. ])

Die Ertragsfunktionen werden in Abhängigkeit der Produkte, der Ressourcenkapazität und des Buchungshorizonts ermittelt:

In [18]:
matrix = np.array([[[2, 2],2], [[2, 1],1], [[2, 0],0], [[1, 2],2], [[0, 1],1], [[0, 2], 0]])
matrix[0][0][0]

2

In [19]:
from functools import wraps
def Memo(func):
    '''A Memoizing Decorator. The idea of memoized function is that it caches its return values. If you call it a
    second time with the same parameters, it will simply return the cached value. You can certainly (allerdings) 
    put this sort of caching logic inside your function, but memo function is a more reusable (Mehrweg-) solution.
    '''
    cache = {}                           # Stored subproblem solutions
    @wraps(func)                         # Make wrap look like func
    def wrap(*args):                     # The memoized wrapper
        if args not in cache:            # Not already computed?
            cache[args] = func(*args)    # Compute & cache the solution
        return cache[args]               # Return the cached solution
    return wrap  

In [20]:
print("Produkte j:", products)            
print("Ressourcen h:", resources)         
print("Kapazitaet c[h]:", capacities)           
print("Verbrauch a[j]:", consumtions)       
print("Ertrag r[j]:", revenues)         
print("Buchungszeit t:", times)             
print( "Wkeit der Anfrage p[j, t]:", probs)             
print("Wkeit keiner Anfrage p[0, t]", against_probs)    

('Produkte j:', array([0, 1, 2, 3]))
('Ressourcen h:', array([0, 1, 2]))
('Kapazitaet c[h]:', array([ 0.,  5.,  4.]))
('Verbrauch a[j]:', array([[ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  1.]]))
('Ertrag r[j]:', array([   0.,  100.,  200.,  300.]))
('Buchungszeit t:', array([4, 3, 2, 1, 0]))
('Wkeit der Anfrage p[j, t]:', array([[ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.4,  0.3,  0.2,  0.1,  0. ],
       [ 0.3,  0.2,  0.1,  0.2,  0. ],
       [ 0.2,  0.1,  0.2,  0.3,  0. ]]))
('Wkeit keiner Anfrage p[0, t]', array([ 0.1,  0.4,  0.5,  0.4,  1. ]))


Beim nachfolgenden Code komme ich nicht weiter... Die Rekursion macht keinen Spass...

In [21]:
conditions = []
for i in resources[1:]:
    for k in consumtions[i:]:
        for t in range(len(times)):
            conditions.append([(capacities-consumtions[1]), times[t]])
        
conditions

[[array([ 0.,  4.,  4.]), 4],
 [array([ 0.,  4.,  4.]), 3],
 [array([ 0.,  4.,  4.]), 2],
 [array([ 0.,  4.,  4.]), 1],
 [array([ 0.,  4.,  4.]), 0],
 [array([ 0.,  4.,  4.]), 4],
 [array([ 0.,  4.,  4.]), 3],
 [array([ 0.,  4.,  4.]), 2],
 [array([ 0.,  4.,  4.]), 1],
 [array([ 0.,  4.,  4.]), 0],
 [array([ 0.,  4.,  4.]), 4],
 [array([ 0.,  4.,  4.]), 3],
 [array([ 0.,  4.,  4.]), 2],
 [array([ 0.,  4.,  4.]), 1],
 [array([ 0.,  4.,  4.]), 0],
 [array([ 0.,  4.,  4.]), 4],
 [array([ 0.,  4.,  4.]), 3],
 [array([ 0.,  4.,  4.]), 2],
 [array([ 0.,  4.,  4.]), 1],
 [array([ 0.,  4.,  4.]), 0],
 [array([ 0.,  4.,  4.]), 4],
 [array([ 0.,  4.,  4.]), 3],
 [array([ 0.,  4.,  4.]), 2],
 [array([ 0.,  4.,  4.]), 1],
 [array([ 0.,  4.,  4.]), 0]]

In [22]:
import matplotlib.pyplot as plt
import numpy as np
a = np.linspace(0,10,100)
b = np.exp(-a)
plt.plot(a,b)
plt.show()

ImportError: dlopen(/Users/Superuser/anaconda/lib/python2.7/site-packages/matplotlib/ft2font.so, 2): Library not loaded: libfreetype.6.dylib
  Referenced from: /Users/Superuser/anaconda/lib/python2.7/site-packages/matplotlib/ft2font.so
  Reason: Incompatible library version: ft2font.so requires version 18.0.0 or later, but libfreetype.6.dylib provides version 17.0.0

In [23]:
def test_value(products, capacities, consumtions, revenues, probs, times):
    cache = []
    for i in products[1:]:
        for t in times:
            if times.any()==0:
                reject = 0
                continue
            else:
                reject = test_value(products, capacities, consumtions, revenues, probs[t+1:], times[t+1:])
            if times.any()==0:
                accept = 0 
                continue
            else:    
                accept = test_value(products, capacities-consumtions[i], consumtions, revenues, probs[t+1:], times[t+1:])
            value = reject #+ sum(probs[i][t] * (revenues[i] - reject + accept) for i in products[1:])
            cache.append(value)
            print cache
    return



test_value(products, capacities, consumtions, revenues, probs, times)

[None]
[None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None]
[None]
[None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None]
[None, None]
[None, None, None]
[None, None, None, None]
[None, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None, None, None, None, 

In [24]:
def test_value(products, capacities, consumtions, revenues, probs, times):
    accept = np.zeros(shape=(30,len(times)))
    for i in products[1:]:
        for t in range(len(times)):
            if all(capacities[1:])==True: #Prüft nur, ob alle Zahlen 0 sind..
                if any(times):
                    new_times = times[t+1:]
                else:
                    break
                print(new_times)
                new_capacities = capacities-consumtions[i+1]
                print(new_capacities)
                accept[i+1][t] = probs[i+1][t] * ( revenues[i+1] + test_value(products, new_capacities, 
                                                                     consumtions, revenues, probs, new_times))
            else:
                break
            break
        break
    return accept


test_value(products, capacities, consumtions, revenues, probs, times)

[3 2 1 0]
[ 0.  5.  3.]
[2 1 0]
[ 0.  5.  2.]
[1 0]
[ 0.  5.  1.]
[0]
[ 0.  5.  0.]


ValueError: setting an array element with a sequence.

In [51]:
Value(products, capacities, consumtions, revenues, times, probs)

NameError: name 'Value' is not defined

In [52]:
def value(c, t):
    matrix = np.array = []
    matrix = np.zeros(shape=(len(c+1),len(t)))
    for i in range(1, len(c)):
        for j in range(0, len(t)):
            matrix[i][j] = np.zeros(shape=2)
            matrix[i][j][0] = c[i]
            matrix[i][j][1] = t[j]
    return matrix

value(capacities, times)

ValueError: setting an array element with a sequence.

In [123]:
#@Memo
def Value(Product, Resource, Capacity, Consumption, Revenue, Time, Prob, Against_Prob):
    #accept = np.zeros(len(j)+1)                               # Funktioniert
    #reject = np.zeros(len(j)+1)                               # Funktioniert
    accept = []
    reject = []
    for i in Product:                                   # 0 -> 1 ; 1 -> 2 ; usw.
        for t in range(len(Time)):
            accept.append(Prob[i+1][t] * ( Revenue[i+1] + Value(Product, Resource, Capacity-Consumption[i], 
                                                                Consumption, Revenue, Time[t+1:], Prob, Against_Prob) ))
            reject.append(Prob[i+1][t] * Value(j==i+1, c[i], a[i], r[i], t-1, p[i]))
        if reject[i] > accept[i]: #Was mit größer gleich?
            return reject
        else:
            return accept

Value(products, resources, capacities, consumtions, revenues, times, probs, against_probs)

IndexError: list index out of range

In [79]:
for i in range(30):
    for t in times:
        accept += np.array([i, t])
print accept


TypeError: can only concatenate list (not "int") to list

In [53]:
times
chart = pydecode.ChartBuilder(times)
chart.init(times[1:len(times)])
chart.set(times[0],
          [[times[1]], [times[2]], [times[3]], [times[4]]],
          labels=[1,2,3,4])
pydecode.draw(chart.finish(), labels=True)

TypeError: 'list' object is not callable

In [54]:
def simple_hypergraph():
    """
    Create a simple hypergraph.
    """
    enc = np.arange(7)                                          # array([0, 1, 2, 3, 4, 5])
    c = pydecode.ChartBuilder(enc, np.arange(10))               # <pydecode._pydecode.ChartBuilder at 0x105604150>

    c.init(enc[:4])                                             # None
    
    c.set_t(enc[4], enc[0:2], enc[1:3], labels=np.arange(2))    # None
    
    c.set_t(enc[5], np.repeat(enc[4], 1), enc[[3]], labels=np.array([2])) # None
    
    c.set_t(enc[6], np.repeat(enc[5], 1), enc[[4]], labels=np.array([3])) # None
    
    dp = c.finish()                                             # <pydecode._pydecode.Hypergraph object at 0x1079d2aa0>
    # for edge in hypergraph.edges:
    #     assert edge.label in ["0", "1", "2", "3", "4"]
    return dp

hypergraph = simple_hypergraph()

for vertex in hypergraph.vertices:
    print vertex.id, vertex.is_terminal
    for edge in vertex.edges:
        print "\t", edge.id, edge.label

TypeError: 'list' object is not callable

In [ ]:
def DynPro(capacity, time):
    